## IMPORTS

In [1]:
from facebook_page_scraper import Facebook_scraper
from dotenv import load_dotenv
import os
from selenium import webdriver
from selenium.webdriver.firefox.service import Service

## WEB CONFIGS

In [2]:
# Specify the correct Geckodriver path
GECKODRIVER_PATH = "/usr/local/bin/geckodriver"  # Adjust as needed

# Configure Firefox options
options = webdriver.FirefoxOptions()
options.binary_location = "/Applications/Firefox.app/Contents/MacOS/firefox"
# options.add_argument("--headless")  # Optional: Remove this line if you want to see the browser

# Start Selenium WebDriver manually
service = Service(GECKODRIVER_PATH)
driver = webdriver.Firefox(service=service, options=options)

## LOGINS

In [3]:
# Load .env file
load_dotenv("logins.env")

# Facebook credentials from environment variables
fb_email = os.getenv('FACEBOOK_EMAIL')
fb_password = os.getenv('FACEBOOK_PASSWORD')

# Quick check
print(f"Email loaded: {fb_email is not None}, Password loaded: {fb_password is not None}")

Email loaded: True, Password loaded: True


## SCRAPER SETTINGS

In [4]:
# Facebook scraping parameters
page_group_or_name = 1996906587229548
posts_count = 10  # Adjust as needed
browser = "firefox"
proxy = None
timeout = 600 # Seconds, important bc you need to login during window
headless = False  # Should match WebDriver options, please do this so you can log in
isGroup = True

## RUN SCRAPER

In [5]:
scraper = Facebook_scraper(
    page_group_or_name,
    posts_count,
    browser,
    proxy=proxy,
    timeout=timeout,
    headless=headless,
    isGroup=isGroup,
    username=fb_email,
    password=fb_password
)

In [8]:
json_data = scraper.scrap_to_json()

2025-03-30 22:50:04,351 - facebook_page_scraper.element_finder - ERROR - Error at login: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:197:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:527:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16
Traceback (most recent call last):
  File "/Users/aleezarao/.local/lib/python3.12/site-packages/facebook_page_scraper/element_finder.py", line 505, in __login
    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/aleezarao/.local/lib/python3.12/site-packages/selenium/webdriver/support/wait.py", line 89, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Mes

new layout loaded


2025-03-30 22:50:29,782 - facebook_page_scraper.driver_utilities - ERROR - Error at close_modern_layout_signup_modal: Message: Element <div class="x1i10hfl xjqpnuy xa49m3k xqeqjp1 x2hbi6w x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x1ypdohk xdl72j9 x2lah0s xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r x2lwn1j xeuugli x16tdsg8 x1hl2dhg xggy1nq x1ja2u2z x1t137rt x1q0g3np x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xzolkzo x12go9s9 x1rnf11y xprq8jg x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x78zum5 xl56j7k xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 x10w6t97 x1td3qas x1qhmfi1"> could not be scrolled into view
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:197:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:359:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:167:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:136:11

In [9]:
print(json_data)

{"100064901131523": {"name": "Justice for Otsego", "content": "Hey everyone! Just wanted to give an update. We are working to create JFO as a nonprofit. It’s currently in the works and coming soon. We are still working with the Otsego Story Project team through MSU and working on lots of plans for that. Stay tuned and I will give more detailed updates as I learn more. Thanks for sticking with us. Christina", "images": [], "post_url": "https://www.facebook.com/groups/1996906587229548/user/100064901131523/"}, "1335492083": {"name": "Cinthia Williams", "content": "", "images": ["https://scontent-det1-1.xx.fbcdn.net/v/t39.30808-6/482247780_10236038111756513_2952223678371818838_n.jpg?stp=c361.0.2496.1303a_dst-jpg_s1080x2048_tt6&_nc_cat=101&ccb=1-7&_nc_sid=75d36f&_nc_ohc=uWLWcCjqtA8Q7kNvgHeQiFo&_nc_oc=Admyzo8PANceL_JnFQ5ThZhdhmXVkGJwKrThlp7PwIlvzLqpdSQrMLDD8ydpowfEmgo&_nc_zt=23&_nc_ht=scontent-det1-1.xx&_nc_gid=-yPeHUW9GsOWCT0B9MS2Ug&oh=00_AYGs4HjPvxs6yfxab61sVP_zAVcCMAaeZiC0Lqgh3wT-qw&oe=67

In [10]:
import json
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
data = json.loads(json_data)

posts = []
for key, value in data.items():
    content = value.get("content", "").strip()
    posts.append(content)

sia = SentimentIntensityAnalyzer()

sentiment_results = []
positive_count = 0
negative_count = 0
neutral_count = 0

print("Individual Post Sentiment:\n")

for post in posts:
    scores = sia.polarity_scores(post)
    sentiment_results.append(scores)
    
    print(f"Post: {post}")
    print(f"Scores: {scores}\n")
    
    compound = scores['compound']
    if compound >= 0.05:
        positive_count += 1
    elif compound <= -0.05:
        negative_count += 1
    else:
        neutral_count += 1

n_posts = len(sentiment_results)
if n_posts > 0:
    avg_compound = sum(s['compound'] for s in sentiment_results) / n_posts
    avg_pos = sum(s['pos'] for s in sentiment_results) / n_posts
    avg_neg = sum(s['neg'] for s in sentiment_results) / n_posts
    avg_neu = sum(s['neu'] for s in sentiment_results) / n_posts

    print("---------------------------------------------------------")
    print(f"Total posts analyzed: {n_posts}")
    print(f"Positive posts: {positive_count}")
    print(f"Negative posts: {negative_count}")
    print(f"Neutral posts:  {neutral_count}")
    print("\nOverall Average Sentiment Scores (across all posts):")
    print(f"  Compound: {avg_compound:.3f}")
    print(f"  Positive: {avg_pos:.3f}")
    print(f"  Negative: {avg_neg:.3f}")
    print(f"  Neutral:  {avg_neu:.3f}")
    print("\nConclusion:")
    if avg_compound >= 0.05:
        print("Overall sentiment is generally **positive**.")
    elif avg_compound <= -0.05:
        print("Overall sentiment is generally **negative**.")
    else:
        print("Overall sentiment is **neutral or mixed**.")

else:
    print("No valid posts found for sentiment analysis.")
     
 

Individual Post Sentiment:

Post: Hey everyone! Just wanted to give an update. We are working to create JFO as a nonprofit. It’s currently in the works and coming soon. We are still working with the Otsego Story Project team through MSU and working on lots of plans for that. Stay tuned and I will give more detailed updates as I learn more. Thanks for sticking with us. Christina
Scores: {'neg': 0.0, 'neu': 0.914, 'pos': 0.086, 'compound': 0.6793}

Post: 
Scores: {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

Post: Does anyone here live on Helen Ave. in Otsego that would be willing to let myself and some people working on the Otsego Story Project from MSU get a water sample from your tap that does not have reverse osmosis in your home? We would need to get it between 3-4pm tomorrow on Sunday the 23rd. Sorry this is last minute. Anyone willing to help would be greatly appreciated. If you do not want to volunteer here please feel free to contact me via private message. Thank you so

In [15]:
# Store labeled posts
labeled_posts = []

for post in posts:
    scores = sia.polarity_scores(post)
    compound = scores['compound']
    if compound >= 0.05:
        sentiment = 'positive'
    elif compound <= -0.05:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    
    labeled_posts.append({'post': post, 'sentiment': sentiment, **scores})


In [16]:
# Convert to DataFrame
import pandas as pd
df = pd.DataFrame(labeled_posts)

In [18]:
# Save to CSV
import os
os.makedirs("data", exist_ok=True)
df.to_csv("data/fb_sentiment_results.csv", index=False)